In [1]:
# same setup

import os

from dotenv import load_dotenv

load_dotenv()

key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

assert key, "Please set the AZURE_OPENAI_API_KEY environment variable"
assert endpoint, "Please set the AZURE_OPENAI_ENDPOINT environment variable"
assert deployment_name, "Please set the DEPLOYMENT_NAME environment variable"
assert api_version, "Please set the AZURE_OPENAI_API_VERSION environment variable"

SYS_MSG = "You are a synonym and antonym bot. Given a word, you will provide synonyms and antonyms for it."

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path

loader = PyPDFLoader(str(Path("insurance.pdf")))
pages = loader.load_and_split()

In [20]:
len(pages)

5

In [23]:
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings

# deployment name for an embeddings model =
embedding_deployment = "ss_embed"

faiss_index = FAISS.from_documents(
    pages,
    AzureOpenAIEmbeddings(deployment=embedding_deployment),
)
docs = faiss_index.similarity_search("Car", k=1)

In [24]:
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

2: Vehicle Damage Coverage
Coverage details for vehicle repair or replacement due to accidents or theft.
1. Collision: Coverage of repair costs 100% up to the value of $40,000.
2. Theft: Full replacement cost up to $50,000 if the vehicle is stolen and not recovered.
3. Natural Disasters: 70% coverage f


In [26]:
query = "flood"
docs = faiss_index.similarity_search(query)
print(f"found {len(docs)} documents")
print("top hit:")
print(docs[0].page_content)

found 4 documents
top hit:
Vehicle Damage Coverage
Coverage details for vehicle repair or replacement due to accidents or theft.
1. Collision: Coverage of repair costs 100% up to the value of $40,000.
2. Theft: Full replacement cost up to $50,000 if the vehicle is stolen and not recovered.
3. Natural Disasters: 70% coverage for damages caused by events such as earthquakes, floods,
and hurricanes up to $30,000.
4. Vandalism: Coverage up to $10,000.
Exclusions:
- Damages due to routine wear and tear.
- Incidents occurring when the vehicle was being used for illegal activities.
- Vehicles over 10 years old or those not maintaining regular servicing records.


In [30]:
retriever = faiss_index.as_retriever(k=1)

In [35]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

prompt = """ You are a helpful insurance agent. Given the context from the insurance policy, shortly answer the user's question.
Please provide extensive evidence from the document to support your answer.
<policy>
{context}
</policy>
question = {question}
if the answer is not in the <policy> provided, please share what you do understand with the policy provided, 
and let the user know it may not be exactly what they are looking for."
"""

prompt = ChatPromptTemplate.from_template(prompt)


llm = AzureChatOpenAI(
    api_version=api_version,  # type: ignore
    azure_deployment=deployment_name,
)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [36]:
user_prompt = "car collision"

rag_chain.invoke(user_prompt)

'According to the insurance policy document, there is coverage for car collision damages. The policy provides 100% coverage for repair costs up to the value of $40,000 under the Collision section. Here is the evidence from the document: "1. Collision: Coverage of repair costs 100% up to the value of $40,000." Therefore, the insurance policy covers car collision damages up to $40,000.'